# ECCO Ocean Mixed Layer Energy Budget 

The following sample code verifies that the ocean mixed layer energy budget is balanced in an example region, 
i.e., the ocean heat storage tendency is equal to the sum of surface heat fluxes 
plus ocean heat convergence (advection, diffusion), less surface radiation out the bottom.

In [1]:
import xarray as xr
import gcsfs
import intake
import numpy as np
import matplotlib
#import cmocean
import stats
import stats as st
from matplotlib import pyplot as plt
%matplotlib inline

/srv/conda/envs/notebook/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


Set up plotting parameters.

In [2]:
matplotlib.rcParams.update({'font.size': 16})
matplotlib.rcParams.update({'axes.titlesize': 16})
matplotlib.rcParams.update({'figure.figsize': (10,8)})
matplotlib.rcParams.update({'lines.linewidth': 2})
matplotlib.rcParams.update({'legend.fontsize': 18})
matplotlib.rcParams.update({'mathtext.fontset': 'cm'})
matplotlib.rcParams.update({'ytick.major.size': 3})
matplotlib.rcParams.update({'axes.labelsize': 16})
matplotlib.rcParams.update({'ytick.labelsize': 16})
matplotlib.rcParams.update({'xtick.labelsize': 16})

Open the Pangeo data catalog and load the **ECCOv4r3** product as an xarray Dataset. The dataset is *lazily* evaluated, which is a good thing considering the size is around 100 GB.  

In [3]:
cat = intake.open_catalog('catalog.yaml')
list(cat)

['sea_surface_height',
 'ECCOv4r3',
 'SOSE',
 'LLC4320_grid',
 'LLC4320_SST',
 'LLC4320_SSS',
 'LLC4320_SSH',
 'LLC4320_SSU',
 'LLC4320_SSV',
 'CESM_POP_hires_control',
 'CESM_POP_hires_RCP8_5',
 'GFDL_CM2_6_control_ocean_surface',
 'GFDL_CM2_6_control_ocean_3D',
 'GFDL_CM2_6_one_percent_ocean_surface',
 'GFDL_CM2_6_one_percent_ocean_3D',
 'GFDL_CM2_6_grid']

In [4]:
ds = cat.ECCOv4r3.to_dask()
ds

<xarray.Dataset>
Dimensions:    (face: 13, i: 90, i_g: 90, j: 90, j_g: 90, k: 50, k_l: 50, k_p1: 51, k_u: 50, time: 288, time_snp: 287)
Coordinates:
    Depth      (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    PHrefC     (k) float32 dask.array<shape=(50,), chunksize=(50,)>
    PHrefF     (k_p1) float32 dask.array<shape=(51,), chunksize=(51,)>
    XC         (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    XG         (face, j_g, i_g) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    YC         (face, j, i) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    YG         (face, j_g, i_g) float32 dask.array<shape=(13, 90, 90), chunksize=(13, 90, 90)>
    Z          (k) float32 dask.array<shape=(50,), chunksize=(50,)>
    Zl         (k_l) float32 dask.array<shape=(50,), chunksize=(50,)>
    Zp1        (k_p1) float32 dask.array<shape=(51,), chunksize=(51,)>
    Zu         (k_u) float32 dask.array<

In [5]:
ds.nbytes / 1e12

0.134190544604

Drop the coordinates from the main dataset; a trick to make things work a bit faster.

In [7]:
coords = ds.coords.to_dataset().reset_coords()
ds = ds.reset_coords(drop=True)

Load ECCO mixed layer depth from my **Google Cloud Bucket**. Data can be easily and inexpensively be stored on the **Google cloud** here: https://cloud.google.com/

In [8]:
ds_MXLDEPTH = xr.open_zarr(gcsfs.GCSMap('ecco-data/MXLDEPTH'))
mxldepth = ds_MXLDEPTH.MXLDEPTH

For some reason the mixed layer depth coordinate indices are displaced by +1 in relation to the ECCO data stored on Pangeo. The coordinates need to be matched for future calculations. 

In [9]:
mxldepth.coords['i'] = coords['i']
mxldepth.coords['j'] = coords['j']
mxldepth

<xarray.DataArray 'MXLDEPTH' (face: 13, time: 288, j: 90, i: 90)>
dask.array<shape=(13, 288, 90, 90), dtype=float64, chunksize=(1, 288, 90, 90)>
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
    lat      (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 90, 90)>
    lon      (face, j, i) float64 dask.array<shape=(13, 90, 90), chunksize=(1, 90, 90)>
    tim      (time) datetime64[ns] dask.array<shape=(288,), chunksize=(288,)>
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 285.0 286.0 287.0 288.0
Dimensions without coordinates: face
Attributes:
    long_name:  Mixed-Layer Depth (>0)
    units:      m

Calculate climatological mean mixed layer depth. We will be using this later to mask grid points outside of the mixed layer. 

In [10]:
mxldepth_clim=mxldepth.mean(dim='time').load()

Import the plotting library. This will help with mapping fields from the LLC90 grid that ECCO utilizes (see https://www.geosci-model-dev.net/8/3071/2015/gmd-8-3071-2015.pdf) to a standard latitude-longitude grid. 

In [11]:
import importlib
import llcmapping
importlib.reload(llcmapping)
from llcmapping import LLCMapper

`mapper` is the object we will use to plot fields later.

In [12]:
mapper = LLCMapper(coords)

Let's test how long it takes to load sea-surface temperature (SST) into memory.

In [13]:
%time ds.THETA.isel(k=0).mean(dim='time').load()

CPU times: user 44.6 s, sys: 27.6 s, total: 1min 12s
Wall time: 37.3 s


<xarray.DataArray 'THETA' (face: 13, j: 90, i: 90)>
array([[[ 0.      ,  0.      , ...,  0.      ,  0.      ],
        [ 0.      ,  0.      , ...,  0.      ,  0.      ],
        ...,
        [ 0.432548,  0.420362, ...,  0.243915,  0.291386],
        [ 0.653515,  0.658455, ...,  0.427649,  0.498188]],

       [[ 0.872101,  0.891548, ...,  0.602339,  0.697784],
        [ 1.096168,  1.117599, ...,  0.774091,  0.891288],
        ...,
        [27.589699, 27.559296, ..., 26.44383 , 26.62003 ],
        [27.37777 , 27.343906, ...,  0.      , 26.430416]],

       ...,

       [[27.479862, 27.624815, ...,  5.60189 ,  5.157408],
        [27.488213, 27.622663, ...,  5.657171,  5.213162],
        ...,
        [27.444382, 27.641308, ...,  1.204041,  1.031747],
        [27.411293, 27.615599, ...,  1.113161,  0.912151]],

       [[ 4.696424,  4.21946 , ...,  0.      ,  0.      ],
        [ 4.747999,  4.27007 , ...,  0.      ,  0.      ],
        ...,
        [ 0.851896,  0.651258, ...,  0.      ,  0. 

`xgcm` is a python package for working with the datasets produced by numerical General Circulation Models (GCMs) and similar gridded datasets that are amenable to finite volume analysis. This will come in handy later when performing calculus on the LLC90 grid.

In [89]:
import xgcm

# define the connectivity between faces
face_connections = {'face':
                    {0: {'X':  ((12, 'Y', False), (3, 'X', False)),
                         'Y':  (None,             (1, 'Y', False))},
                     1: {'X':  ((11, 'Y', False), (4, 'X', False)),
                         'Y':  ((0, 'Y', False),  (2, 'Y', False))},
                     2: {'X':  ((10, 'Y', False), (5, 'X', False)),
                         'Y':  ((1, 'Y', False),  (6, 'X', False))},
                     3: {'X':  ((0, 'X', False),  (9, 'Y', False)),
                         'Y':  (None,             (4, 'Y', False))},
                     4: {'X':  ((1, 'X', False),  (8, 'Y', False)),
                         'Y':  ((3, 'Y', False),  (5, 'Y', False))},
                     5: {'X':  ((2, 'X', False),  (7, 'Y', False)),
                         'Y':  ((4, 'Y', False),  (6, 'Y', False))},
                     6: {'X':  ((2, 'Y', False),  (7, 'X', False)),
                         'Y':  ((5, 'Y', False),  (10, 'X', False))},
                     7: {'X':  ((6, 'X', False),  (8, 'X', False)),
                         'Y':  ((5, 'X', False),  (10, 'Y', False))},
                     8: {'X':  ((7, 'X', False),  (9, 'X', False)),
                         'Y':  ((4, 'X', False),  (11, 'Y', False))},
                     9: {'X':  ((8, 'X', False),  None),
                         'Y':  ((3, 'X', False),  (12, 'Y', False))},
                     10: {'X': ((6, 'Y', False),  (11, 'X', False)),
                          'Y': ((7, 'Y', False),  (2, 'X', False))},
                     11: {'X': ((10, 'X', False), (12, 'X', False)),
                          'Y': ((8, 'Y', False),  (1, 'X', False))},
                     12: {'X': ((11, 'X', False), None),
                          'Y': ((9, 'Y', False),  (0, 'X', False))}}}

# create the grid object
grid = xgcm.Grid(ds, periodic=False, face_connections=face_connections)

Compute the vertical grid spacing and grid volume, and then load the required fields for the energy budget. Conservation of heat requires the following equation:


$\large
\begin{align}
\frac{\partial (s^{*} \theta)}{\partial t} = -\nabla_{z^{*}} \cdot (s^{*} \mathbf{u} \theta) - \frac{\partial (s^{*}w \theta)}{\partial z} +   s^{*}\mathit{D}_\theta + \frac{s^{*}}{\rho c_p} \mathit{F}_\theta 
\end{align}$


Here, $s^{*} = 1 + \eta / H$ is a scale factor, which arises because the equations are expressed in terms of a re-scaled height coordinate:

$\large z^{*} = \frac{ z - \eta (x,y,t)}{H(x,y) + \eta (x,y,t)} H(x,y)$

where $\eta$ is the surface height and $H$ is the ocean depth.

In the conservation of heat equation, $\theta$ is the potential temperature, $\mathbf{u}$ and $w$ are total horizontal and vertical velocities, respectively, $\mathit{D_\theta}$ is the heat flux convergence due to diffusive procesess, and $\mathit{F_\theta}$ is the surface forcing (sum of latent, sensible and radiative heat fluxes) and geothermal forcing term. 

Load the required monthly fields.

In [65]:
# First test without subtracting climatological mean, and test for a single face

# Load monthly temperature snapshot
THETAsnp_anom = ds.THETA_snp.isel(face=1)
THETAsnp_anom = THETAsnp_anom.rename({'time_snp':'time'})

# Load monthly surface height snapsnot
ETANsnp_anom = ds.ETAN_snp.isel(face=1)
ETANsnp_anom = ETANsnp_anom.rename({'time_snp':'time'})

# Load monthly mean fields
# Surface heat flux
Qs_anom = ds.TFLUX.isel(face=1)
oceQsw_anom = ds.oceQsw.isel(face=1)

# Advection
ADVx_TH_anom = ds.ADVx_TH.isel(face=1)
ADVy_TH_anom = ds.ADVy_TH.isel(face=1)
ADVr_TH_anom = ds.ADVr_TH.isel(face=1)

# Diffusion 
DFxE_TH_anom = ds.DFxE_TH.isel(face=1)
DFyE_TH_anom = ds.DFyE_TH.isel(face=1)
DFrE_TH_anom = ds.DFrE_TH.isel(face=1)
DFrI_TH_anom = ds.DFrI_TH.isel(face=1)

Before doing the budget calculations, a few parameters need to be defined:

In [47]:
dt = coords.time_snp[1:].load()
dt = dt.rename({'time_snp': 'time'})
# delta t in seconds. Note: divide by 10**9 to convert nanoseconds to seconds
dt.values = [float(t)/10**9 for t in np.diff(coords.time_snp)]

# time axis of dt should be the same as of the monthly averages
dt.time.values = coords.time[1:-1].values

In [14]:
rho0 = 1029 #sea-water density (kg/m^3)
c_p = 3994 #sea-water heat capacity (J/kg/K)

# Constants for surface heat penetration (from Table 2 of Paulson and Simpson, 1977)
R = 0.62
zeta1 = 0.6
zeta2 = 20.0

In [17]:
# Ocean depth (m)
Depth = coords.Depth.load()

In [22]:
dxG = coords.dxG.load()
dyG = coords.dyG.load()

In [23]:
# Vertical grid spacing
drF = coords.drF.load()
# Grid volume
vol = coords.rA*coords.drF*coords.hFacC

In [77]:
# Make copy of hFacC
mskC = coords.hFacC.isel(face=1).copy(deep=True).load()

# Change all fractions (ocean) to 1. land = 0
mskC.values[mskC.values>0] = 1

In [25]:
# Make 2D land mask for surface (This is just for plotting/mapping purposes)
land_mask = mskC[0]
land_mask.values[land_mask.values==0] = np.nan

## Evaluating the heat budget

### Total Tendency

In [52]:
# Calculate the s*theta term 
sstar = (1+ETANsnp_anom/Depth)
HCsnp = THETAsnp_anom*sstar #note that this has units of K and is very close to being equal to theta (sstar is small)

In [53]:
# Total tendency (degC/month)
tendH_perMonth = (HCsnp.shift(time=-1)-HCsnp)[:-1]

In [54]:
# Make sure time axis is the same as for the monthly variables
tendH_perMonth.time.values = ds.time[1:-1].values

In [57]:
# Convert tendency from 1/month to 1/s
tendH_perSec = tendH_perMonth/dt

In [58]:
# Predefine tendH array with correct dimensions
tendH = xr.DataArray(np.nan*np.zeros([np.shape(tendH_perSec)[0]+2,50,90,90]),
                     coords={'time': range(np.shape(tendH_perSec)[0]+2),'k': np.array(range(0,50)),
                             'j': np.array(range(0,90)),'i': np.array(range(0,90))},dims=['time','k','j','i'])

In [63]:
tendH.time.values = ds.time.values
# Add coordinates
tendH['XC'] = coords.XC.sel(face=1)
tendH['YC'] = coords.YC.sel(face=1)
tendH['Z'] = coords.Z

### Forcing 

In [66]:
Z = coords.sel(face=1).Z.load()
RF = np.concatenate([coords.sel(face=1).Zp1.values[:-1],[np.nan]])

In [67]:
q1 = R*np.exp(1.0/zeta1*RF[:-1]) + (1.0-R)*np.exp(1.0/zeta2*RF[:-1])
q2 = R*np.exp(1.0/zeta1*RF[1:]) + (1.0-R)*np.exp(1.0/zeta2*RF[1:])

In [68]:
# Correction for the 200m cutoff
zCut = np.where(Z < -200)[0][0]
q1[zCut:] = 0
q2[zCut-1:] = 0

In [69]:
# Save q1 and q2 as xarray data arrays
q1 = xr.DataArray(q1,coords=[Z.k],dims=['k'])
q2 = xr.DataArray(q2,coords=[Z.k],dims=['k'])

In [76]:
mskC

<xarray.DataArray 'hFacC' (k: 50, face: 13, j: 90, i: 90)>
array([[[[nan, ..., nan],
         ...,
         [ 1., ...,  1.]],

        ...,

        [[ 1., ..., nan],
         ...,
         [ 1., ..., nan]]],


       ...,


       [[[ 0., ...,  0.],
         ...,
         [ 0., ...,  0.]],

        ...,

        [[ 0., ...,  0.],
         ...,
         [ 0., ...,  0.]]]], dtype=float32)
Coordinates:
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48 49
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
Attributes:
    long_name:      vertical fraction of open cell
    standard_name:  cell_vertical_fraction

#### Compute vertically penetrating shortwave flux

In [94]:
# Surface heat flux (below the surface)
forcH = ((q1*(mskC==1)-q2*(mskC.shift(k=-1)==1))*oceQsw_anom).transpose('time','k','j','i')

# Reset surface layer to zero
forcH.values[:,0] = 0*forcH.values[:,0]

In [95]:
forcH.load()

<xarray.DataArray (time: 288, k: 50, j: 90, i: 90)>
array([[[[126.251106, ..., 138.314294],
         ...,
         [161.145349, ..., 155.371849]],

        ...,

        [[  0.      , ...,   0.      ],
         ...,
         [  0.      , ...,   0.      ]]],


       ...,


       [[[168.273574, ..., 149.655965],
         ...,
         [136.516031, ..., 148.455296]],

        ...,

        [[  0.      , ...,   0.      ],
         ...,
         [  0.      , ...,   0.      ]]]])
Coordinates:
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 40 41 42 43 44 45 46 47 48 49
    face     int64 1
  * i        (i) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * j        (j) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * time     (time) datetime64[ns] 1992-01-15 1992-02-13 ... 2015-12-14

In [96]:
# Surface heat flux (at the sea surface)
forcH[:,0] = ((Qs_anom - (1-(q1[0]-q2[0]))*oceQsw_anom)*mskC[0]).transpose('time','j','i')

#### Load geothermal flux (may add later)

In [84]:
# Create 3d bathymetry mask
mskC_shifted = mskC.shift(k=-1)
mskC_shifted.values[-1,:,:] = 0
mskb = mskC - mskC_shifted

In [86]:
GEOFLX = ds.GEOFLX.isel(face=1) #Can add this to surface forcing term later.

### Advection

#### Horizontal Convergence

In [91]:
# For now, construct grid based on only one face. Will change later. 
grid = xgcm.Grid(ds.sel(face=1), periodic=False)

In [100]:
# Convergence of horizontal advection (degC/s)
adv_hConvH = -(grid.diff(ADVx_TH_anom, 'X', boundary='extend') + 
               grid.diff(ADVy_TH_anom, 'Y', boundary='extend'))/vol

#DF_TH_diff_anom = grid.diff_2d_vector({'X': DFxE_TH_anom, 'Y': DFyE_TH_anom}, boundary='fill')
#DF_TH_conv_anom_vint = -((delz/v)*DF_TH_diff_anom['X']).sum(dim='k')-((delz/v)*DF_TH_diff_anom['Y']).sum(dim='k')

In [102]:
# Convert from degC/s to W/m^2
adv_hConvH = (rho0*c_p*coords.hFacC*drF)*adv_hConvH

#### Vertical Convergence

In [104]:
# Convergence of the vertical advection (degC m^3/s)
adv_vConvH = grid.diff(ADVr_TH_anom, 'Z', boundary='extend')/vol
adv_vConvH.lkoa
adv_vConvH[:,-1,:,:] = -ADVr_TH_anom[:,-1,:,:]

TypeError: this variable's data is stored in a dask array, which does not support item assignment. To assign to this variable, you must first load it into memory explicitly using the .load() method or accessing its .values attribute.

In [ ]:
# Convert from degC/s to W/m^2
adv_vConvH = (rho0*c_p*coords.hFacC*drF)*adv_vConvH

### Diffusion

#### Horizontal Convergence

In [ ]:
# Convergence of horizontal diffusion (degC/s)
dif_hConvH = -(grid.diff(DFxE_TH_anom, 'X', boundary='extend') + 
               grid.diff(DFyE_TH_anom, 'Y', boundary='extend'))/vol

In [ ]:
# Convert from degC/s to W/m^2
dif_hConvH = (rho*c_p*coords.hFacC*drF)*dif_hConvH

#### Vertical Convergence (explicit and implicit)

In [ ]:
# Convergence of vertical diffusion (degC/s)
dif_vConvH = (grid.diff(DFrE_TH_anom, 'Z', boundary='extend') + grid.diff(DFrI_TH_anom, 'Z', boundary='extend'))/vol
dif_vConvH[:,-1,:,:] = -(DFrE_TH_anom + DFrI_TH_anom)[:,-1,:,:]

In [ ]:
# Convert from degC/s to W/m^2
dif_vConvH = (rho*c_p*coords.hFacC*drF)*dif_vConvH

### Total Convergence

In [ ]:
# Total convergence of advective flux
adv_ConvH = adv_hConvH + adv_vConvH

# Total convergence of diffusive flux
dif_ConvH = dif_hConvH + dif_vConvH

# Total convergence
ConvH = adv_ConvH + dif_ConvH

### Sum of Convergence and Forcing

In [97]:
totalH = ConvH + forcH

NameError: name 'ConvH' is not defined